In [1]:
import tensorflow as tf
import numpy as np
import cv2
import os
import numpy as np
import pandas as pd
import imageio

import random
import matplotlib.pyplot as plt
import keras
import tensorflow.keras as K
import tensorflow.keras.backend as Kback

from keras import layers
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Conv2DTranspose
from keras.layers import concatenate
from keras.layers import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.callbacks import EarlyStopping
from keras.applications.resnet import ResNet50
from keras.applications import vgg16
from keras.applications import inception_v3
from keras.src.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications import DenseNet121, NASNetMobile, EfficientNetB0, Xception
from keras.layers import GlobalAveragePooling2D
from keras.applications.densenet import DenseNet121

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter

In [2]:
from zipfile import ZipFile 

# Replace the path to your zip file and the directory where you want to extract the contents
zip_file_path = r"C:\Users\KIIT\Downloads\waste segregation\waste_classification_data.zip"
extracted_dir = r"C:\Users\KIIT\Downloads\waste segregation\extracted_data"# Specify the directory where you want to extract

# Create a zip object and extract the contents to the specified directory
with ZipFile(zip_file_path, 'r') as zObject:
    zObject.extractall(path=extracted_dir)

In [3]:
import os
from PIL import Image

# Define the extracted directory (update this path as needed)
train_dir = r"C:\Users\KIIT\Desktop\waste segregation\extracted_data\DATASET\DATASET\TRAIN"

# Define class names (folder names)
class_names = ['O','R']

# Initialize dictionary to store class shapes
class_shapes = {}

# Iterate over each class and calculate the shape of images
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        shapes = []
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        shapes.append(img.size)  # img.size returns (width, height)
                except Exception as e:
                    print(f"Could not open {file_path}: {e}")
        class_shapes[class_name] = shapes
    else:
        class_shapes[class_name] = []
        print(f"Directory not found: {class_path}")  # Debug print

# Print the shape of images in each class
for class_name, shapes in class_shapes.items():
    print(f"Class {class_name}: {len(shapes)} images with shapes {shapes[:5]} ...")  # Print first 5 shapes for brevity


Directory not found: C:\Users\KIIT\Desktop\waste segregation\extracted_data\DATASET\DATASET\TRAIN\O
Directory not found: C:\Users\KIIT\Desktop\waste segregation\extracted_data\DATASET\DATASET\TRAIN\R
Class O: 0 images with shapes [] ...
Class R: 0 images with shapes [] ...


In [4]:
import os
import cv2

# Define the extracted directory (update this path as needed)
train_dir = r"C:\Users\KIIT\Desktop\waste segregation1\extracted_data\DATASET\DATASET\TRAIN"
resized_dir_train =  r"C:\Users\KIIT\Desktop\waste segregation1\resized_dir1\resized_dir 1.1"

# Create the new directory structure
if not os.path.exists(resized_dir_train):
    os.makedirs(resized_dir_train)

# Define class names (folder names)
class_names = ['O','R']

# Define the target size
target_size = (128, 128)

# Resize images and save them to the new directory
for class_name in class_names:
    class_path = os.path.join(train_dir, class_name)
    resized_class_path = os.path.join(resized_dir_train, class_name)
    
    if not os.path.exists(resized_class_path):
        os.makedirs(resized_class_path)

    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(class_path, filename)
                img = cv2.imread(file_path)
                if img is not None:
                    resized_img = cv2.resize(img, target_size)
                    output_path = os.path.join(resized_class_path, filename)
                    cv2.imwrite(output_path, resized_img)
                else:
                    print(f"Failed to load image: {file_path}")
    else:
        print(f"Directory not found: {class_path}")

print("Image resizing completed.")


Directory not found: C:\Users\KIIT\Desktop\waste segregation1\extracted_data\DATASET\DATASET\TRAIN\O
Directory not found: C:\Users\KIIT\Desktop\waste segregation1\extracted_data\DATASET\DATASET\TRAIN\R
Image resizing completed.


In [5]:
import os
from PIL import Image

resized_dir_train = r"C:\Users\KIIT\Downloads\waste segregation\resized_dir1\resized_dir1.1"
# Define class names (folder names)
class_names = ['O','R']

# Initialize dictionary to store class shapes
class_shapes = {}

# Iterate over each class and calculate the shape of images
for class_name in class_names:
    class_path = os.path.join(resized_dir_train, class_name)
    if os.path.isdir(class_path):
        shapes = []
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        shapes.append(img.size)  # img.size returns (width, height)
                except Exception as e:
                    print(f"Could not open {file_path}: {e}")
        class_shapes[class_name] = shapes
    else:
        class_shapes[class_name] = []
        print(f"Directory not found: {class_path}")  # Debug print

# Print the shape of images in each class
for class_name, shapes in class_shapes.items():
    print(f"Class {class_name}: {len(shapes)} images with shapes {shapes[:5]} ...")  


Class O: 12565 images with shapes [(128, 128), (128, 128), (128, 128), (128, 128), (128, 128)] ...
Class R: 9999 images with shapes [(128, 128), (128, 128), (128, 128), (128, 128), (128, 128)] ...


In [6]:
train_images = []
train_labels = []

# Load images and assign labels
for label, class_name in enumerate(class_names):
    class_path = os.path.join(resized_dir_train, class_name)
    image_files = os.listdir(class_path)

    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read as GBR
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
        img = img / 255.0  # Normalize pixel values (assuming 8-bit images)
        train_images.append(img)
        train_labels.append(label)

In [7]:
print(len(train_images))

22564


In [8]:
import os
from PIL import Image

# Define the extracted directory (update this path as needed)
test_dir = r"C:\Users\KIIT\Downloads\waste segregation\extracted_data\DATASET\DATASET\TEST"

# Define class names (folder names)
class_names = ['O','R']

# Initialize dictionary to store class shapes
class_shapes = {}

# Iterate over each class and calculate the shape of images
for class_name in class_names:
    class_path = os.path.join(test_dir, class_name)
    if os.path.isdir(class_path):
        shapes = []
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        shapes.append(img.size)  # img.size returns (width, height)
                except Exception as e:
                    print(f"Could not open {file_path}: {e}")
        class_shapes[class_name] = shapes
    else:
        class_shapes[class_name] = []
        print(f"Directory not found: {class_path}")  # Debug print

# Print the shape of images in each class
for class_name, shapes in class_shapes.items():
    print(f"Class {class_name}: {len(shapes)} images with shapes {shapes[:5]} ...")  # Print first 5 shapes for brevity


Class O: 1401 images with shapes [(223, 227), (225, 225), (225, 225), (225, 225), (225, 225)] ...
Class R: 1112 images with shapes [(240, 210), (225, 225), (225, 224), (225, 225), (225, 225)] ...


In [9]:
import os
import cv2

# Define the extracted directory (update this path as needed)
test_dir = r"C:\Users\KIIT\Downloads\waste segregation\extracted_data\DATASET\DATASET\TEST"
resized_dir_test = r"C:\Users\KIIT\Downloads\waste segregation\resized_dir1\resized_dir1.2"
# Create the new directory structure
if not os.path.exists(resized_dir_test):
    os.makedirs(resized_dir_test)

# Define class names (folder names)
class_names = ['O','R']

# Define the target size
target_size = (128, 128)

# Resize images and save them to the new directory
for class_name in class_names:
    class_path = os.path.join(test_dir, class_name)
    resized_class_path = os.path.join(resized_dir_test, class_name)
    
    if not os.path.exists(resized_class_path):
        os.makedirs(resized_class_path)

    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if filename.endswith('.jpeg') or filename.endswith('.jpg') or filename.endswith('.png'):
                file_path = os.path.join(class_path, filename)
                img = cv2.imread(file_path)
                if img is not None:
                    resized_img = cv2.resize(img, target_size)
                    output_path = os.path.join(resized_class_path, filename)
                    cv2.imwrite(output_path, resized_img)
                else:
                    print(f"Failed to load image: {file_path}")
    else:
        print(f"Directory not found: {class_path}")

print("Image resizing completed.")


Image resizing completed.


In [10]:
import os
from PIL import Image

resized_dir_test= r"C:\Users\KIIT\Downloads\waste segregation\resized_dir1\resized_dir1.2"

# Define class names (folder names)
class_names = ['O','R']

# Initialize dictionary to store class shapes
class_shapes = {}

# Iterate over each class and calculate the shape of images
for class_name in class_names:
    class_path = os.path.join(resized_dir_test, class_name)
    if os.path.isdir(class_path):
        shapes = []
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            if os.path.isfile(file_path):
                try:
                    with Image.open(file_path) as img:
                        shapes.append(img.size)  # img.size returns (width, height)
                except Exception as e:
                    print(f"Could not open {file_path}: {e}")
        class_shapes[class_name] = shapes
    else:
        class_shapes[class_name] = []
        print(f"Directory not found: {class_path}")  # Debug print

# Print the shape of images in each class
for class_name, shapes in class_shapes.items():
    print(f"Class {class_name}: {len(shapes)} images with shapes {shapes[:5]} ...")  

Class O: 1401 images with shapes [(128, 128), (128, 128), (128, 128), (128, 128), (128, 128)] ...
Class R: 1112 images with shapes [(128, 128), (128, 128), (128, 128), (128, 128), (128, 128)] ...


In [11]:
test_images = []
test_labels = []

# Load images and assign labels
for label, class_name in enumerate(class_names):
    class_path = os.path.join(resized_dir_test, class_name)
    image_files = os.listdir(class_path)

    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)  # Read as GBR
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB) 
        img = img / 255.0  # Normalize pixel values (assuming 8-bit images)
        test_images.append(img)
        test_labels.append(label)

In [12]:
print(len(test_images))

2513


In [13]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=42)

print('Data length:', len(train_images))
print('labels counts:', Counter(train_labels))

train_images = np.array(train_images).reshape(-1, 128, 128, 3)
train_labels = np.array(train_labels)

Data length: 22564
labels counts: Counter({0: 12565, 1: 9999})


In [14]:
test_images, test_labels = shuffle(test_images, test_labels, random_state=42)

print('Data length:', len(test_images))
print('labels counts:', Counter(test_labels))

test_images = np.array(test_images).reshape(-1, 128, 128, 3)

test_labels = np.array(test_labels)

Data length: 2513
labels counts: Counter({0: 1401, 1: 1112})


In [15]:
from sklearn.model_selection import train_test_split

# Combine train and test data
all_images = np.concatenate((train_images, test_images), axis=0)
all_labels = np.concatenate((train_labels, test_labels), axis=0)

# Split the combined data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(all_images, all_labels, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Print the lengths and shapes of the datasets
print('Train length:', len(X_train), X_train.shape)
print('Validation length:', len(X_val), X_val.shape)
print('Test length:', len(X_test), X_test.shape)


Train length: 17553 (17553, 128, 128, 3)
Validation length: 3762 (3762, 128, 128, 3)
Test length: 3762 (3762, 128, 128, 3)


In [16]:
from keras.applications.mobilenet_v2 import MobileNetV2
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(128,128,3)
)

In [17]:
base_model.trainable = False

In [18]:
NUM_CLASSES = len(set(train_labels).union(set(test_labels)))
Model_2 = Sequential()
Model_2.add(base_model)
Model_2.add(MaxPooling2D())
Model_2.add(layers.Flatten())
Model_2.add(layers.Dense(256, activation='relu'))
Model_2.add(layers.Dense(128, activation='relu'))
Model_2.add(layers.Dense(64, activation='relu'))
Model_2.add(layers.Dense(NUM_CLASSES, activation='softmax'))

In [19]:
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
losses = tf.keras.losses.SparseCategoricalCrossentropy()

# Compile the model
Model_2.compile(optimizer=optimizer, loss=losses, metrics=['accuracy'])

In [20]:
Model_2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 max_pooling2d (MaxPooling2  (None, 2, 2, 1280)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 5120)              0         
                                                                 
 dense (Dense)               (None, 256)               1310976   
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [21]:
history_Model_2 = Model_2.fit(X_train,y_train, epochs=20, batch_size=32,validation_data=(X_val,y_val))

Epoch 1/20
549/549 [==============================] - 294s 520ms/step - loss: 0.2479 - accuracy: 0.9053 - val_loss: 0.1689 - val_accuracy: 0.9335
Epoch 2/20
549/549 [==============================] - 267s 486ms/step - loss: 0.1517 - accuracy: 0.9413 - val_loss: 0.1851 - val_accuracy: 0.9234
Epoch 3/20
549/549 [==============================] - 280s 511ms/step - loss: 0.1194 - accuracy: 0.9561 - val_loss: 0.1721 - val_accuracy: 0.9330
Epoch 4/20
549/549 [==============================] - 315s 574ms/step - loss: 0.0832 - accuracy: 0.9680 - val_loss: 0.1875 - val_accuracy: 0.9362
Epoch 5/20
549/549 [==============================] - 283s 516ms/step - loss: 0.0580 - accuracy: 0.9778 - val_loss: 0.2135 - val_accuracy: 0.9365
Epoch 6/20
549/549 [==============================] - 281s 512ms/step - loss: 0.0435 - accuracy: 0.9843 - val_loss: 0.2729 - val_accuracy: 0.9192
Epoch 7/20
549/549 [==============================] - 291s 529ms/step - loss: 0.0337 - accuracy: 0.9880 - val_loss: 0.2783 -

In [22]:
test_loss, test_accuracy = Model_2.evaluate(X_test,y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

118/118 [==============================] - 47s 399ms/step - loss: 0.3108 - accuracy: 0.9333
Test Loss: 0.31076911091804504
Test Accuracy: 0.9332801699638367
